In [1]:
import pandas as pd
import math
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from IPython.display import display

import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [2]:
PATH = "/Users/mustafa/Desktop/bluebook-for-bulldozers/"
df = pd.read_csv(f'{PATH}Train.csv', low_memory = False, parse_dates = ["saledate"])
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.sort_values(by="saledate") #güncel verileri son sıraya aldık. 
# böylece günümüzdeki verileri daha iyi temsil edebilriz. Gerçek hayatı yansıtabiliriz.

### Kategorik Değişkenleri Sayısal Hale Getirmek

In [4]:
df["UsageBand"]

205615    NaN
92803     NaN
98346     NaN
169297    NaN
274835    NaN
         ... 
393123    NaN
397987    NaN
392664    NaN
400458    NaN
400217    NaN
Name: UsageBand, Length: 401125, dtype: object

In [5]:
for n, c in df.items():
    print(n)
    print("-"*10)
    print(c)
    break
#column isimlerini ve değerlerini göstermek için yaptık. break koymasaydık bütün columnlarda yapacaktı. 
#ilk column da yaptı ve break dediğimiz için bitirdi.

SalesID
----------
205615    1646770
92803     1404019
98346     1415646
169297    1596358
274835    1821514
           ...   
393123    6260878
397987    6288376
392664    6258093
400458    6315563
400217    6312170
Name: SalesID, Length: 401125, dtype: int64


In [6]:
#strin dtype olan columnları pandas category tipine çevirir. yani high medium low gibi string olarak ifade edilmişse
#bunu category tipine çevir diyoruz.
def train_categs(df):
    for n, c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()

In [7]:
# train set'e train_categs uygulandıktan sonra aynı category değişimleri olsun diye validation'a ve train'e bu uygulanır
# zira trainde mesela high'a 2 dedik. validation'da buna 0 dersek, sorunlar ortaya çıkar.
def apply_cats(df, train):
    for n, c in df.items():
        if train[n].dtype == "category":
            df[n] = pd.Categorical(c, categories = train[n].cat.categories, ordered = True)
#burada ne dedik? trainde bu kategorik veri tipinde ise, buradaki df'de bu column'u al, pandas kategorik olarak 
#kaydet. Ama categoriler olarak traindeki kategorileri al ki değişiklik olmasın. traindeki maping'i burada da uygula

In [8]:
train_categs(df)

In [9]:
df["UsageBand"] 
#burada bakıyoruz ki bu dönüşümler gerçekleşmemiş ama arka planda gerçekleşti. buna da şöye erişiyoruz.

205615    NaN
92803     NaN
98346     NaN
169297    NaN
274835    NaN
         ... 
393123    NaN
397987    NaN
392664    NaN
400458    NaN
400217    NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

In [10]:
df["UsageBand"].cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [11]:
df["UsageBand"].cat.set_categories(["High", "Medium", "Low"], ordered = True, inplace = True)
# önceden high, low, medium olarak sıralanmıştı. burada bunu düzelttik.

In [12]:
df["UsageBand"].cat.codes #burada dönüşmüş halini görebilriz. output'ta -1ler göreceğiz. bunlar da NaN değerler.

205615   -1
92803    -1
98346    -1
169297   -1
274835   -1
         ..
393123   -1
397987   -1
392664   -1
400458   -1
400217   -1
Length: 401125, dtype: int8

In [13]:
#bu -1'leri sıfır olarak görmek istersek, ya da değiştirmek istersek şöyle yapabiliriz: hepsini 1 yükseltebilriz. 
#böylece artık 0'lar NaN değerler, 1,2,3 ise high medium low.

def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1

In [14]:
numericalize(df, df["UsageBand"], "UsageBand")

In [15]:
df["UsageBand"]

205615    0
92803     0
98346     0
169297    0
274835    0
         ..
393123    0
397987    0
392664    0
400458    0
400217    0
Name: UsageBand, Length: 401125, dtype: int8

In [16]:
#burada veride olan datetime'ı açıcaz. Hangi gün, hangi ay, hangi hafta vs. bunları column olarak eklemek istiyoruz.

def add_datepart(df, dt_name, drop = True):
    dt_column = df[dt_name]
    column_dtype = dt_column.dtype
    
    attr = ['year', 'month', 'week', 'day', 'dayofweek', 'dayofyear', 'is_month_end', 'is_month_start', 
            'is_quarter_end', 'is_quarter_start', 'is_year_end', 'is_year_start']
    
    for a in attr:
        df["Date" + a.capitalize()] = getattr(dt_column.dt, a)
        
    df["Date" + 'Elapsed'] = dt_column.astype(np.int64) // 10 ** 9
    
    #bunları ekledikten sonra, artık bu columns'u atıyor. 
    #zira bu columndan yeterince bilgi çıkarttık buna gerek kalmadı.
    if drop:
        df.drop(dt_name, axis = 1, inplace = True)

In [17]:
add_datepart(df, "saledate")

/Users/mustafa/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # This is added back by InteractiveShellApp.init_path()


In [18]:
df #sonuçta datetime objesini numerik hale dönüştürmüş olduk.

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,DateDay,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start,DateElapsed
205615,1646770,9500,1126363,8434,132,18.0,1974,NaN,0,TD20,...,17,1,17,False,False,False,False,False,False,600998400
92803,1404019,24000,1169900,7110,132,99.0,1986,NaN,0,416,...,31,1,31,True,False,False,False,False,False,602208000
98346,1415646,35000,1262088,3357,132,99.0,1975,NaN,0,12G,...,31,1,31,True,False,False,False,False,False,602208000
169297,1596358,19000,1433229,8247,132,99.0,1978,NaN,0,644,...,31,1,31,True,False,False,False,False,False,602208000
274835,1821514,14000,1194089,10150,132,99.0,1980,NaN,0,A66,...,31,1,31,True,False,False,False,False,False,602208000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393123,6260878,13500,1799594,4102,149,2.0,1000,NaN,0,D4C,...,30,4,364,False,False,False,False,False,False,1325203200
397987,6288376,9750,1872596,4875,149,2.0,1000,NaN,0,520C,...,30,4,364,False,False,False,False,False,False,1325203200
392664,6258093,14500,1877553,3170,149,2.0,1988,NaN,0,580K,...,30,4,364,False,False,False,False,False,False,1325203200
400458,6315563,12500,1869637,26456,149,2.0,2010,NaN,0,L160,...,30,4,364,False,False,False,False,False,False,1325203200


#### Missing Value

In [19]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum:
            df[name+"_null"] = pd.isnull(col)
        df[name] = col.fillna(col.median())
#bu fonksiyon, boş değeri olan bir column varsa, bu column'u alıp, name_null şeklinde yeni bir column oluşturuyor.
#true false şeklinde dolduruyor. daha sonra dönüşüm yaptığı columndaki missing valueları median ile dolduruyor.
#hem missing value sorununu çözüyoruz, hem de önceden missing value oldğunu unutmuyoruz. Bilgi kaybı olmadı.

In [20]:
for n,c in df.items():
    if is_numeric_dtype(c):
        if df[n].isnull().sum():
            print(n)
#burada dönen columnlar, içinde hem numerik değer olup hem de missing value'ları olan columnlar.

auctioneerID
MachineHoursCurrentMeter


In [21]:
df["MachineHoursCurrentMeter"].isnull().sum() #bu kadar missing value var.

258360

In [22]:
fix_missing(df,df["MachineHoursCurrentMeter"], "MachineHoursCurrentMeter")

In [23]:
df["MachineHoursCurrentMeter"].isnull().sum() #artık missing value kalmadı.

0

In [24]:
df["MachineHoursCurrentMeter_null"] #yeni oluşturduğumuz column bu şekilde. böylece hem missing valueları unutmadık,
# hem de missing valueları doldurduk.

205615    True
92803     True
98346     True
169297    True
274835    True
          ... 
393123    True
397987    True
392664    True
400458    True
400217    True
Name: MachineHoursCurrentMeter_null, Length: 401125, dtype: bool

In [25]:
#bu fonksşyona bi df vericez, bir de y_column'u yani tahminlerimiz. önce onu atıp ayrı bir formda tutuyoruz. zira
# tahmin edeceğimiz şeyi girdi olarak girmemesi lazım. zira onun katsayısını 1 yapar ve diğerlerine önem vermez.
def proc_df(df, y_fld):
    
    y= df[y_fld].values
    df.drop([y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
        numericalize(df, c, n)
        
    return df, y

#burada bir hata ile karşılaşabiliriz. nedir bu? train verisetinde missing value olunca mesela x1_null şeklinde bir
# column oluşacak. ama validation setinde bu column yok. hata verecek. ya da tam tersi train setinde böyle bir null değer yok 
# column oluşmayacak ama validation setinde null değer olacak ve x1_null columnu oluşacak. bunu çözmemiz lazım. Nasıl?
# train setinde neleri değiştirdiysek, nelere x1_null eklediysek, bir dictionaryde tutacağız. Ve onun değiştirdim
# değerlerini tutacağız. mesela train setinde x1'in median'ı 50 imiş ve missing valuelar varmış. Diyeceğim ki train setinde x1'i
# null olarak aldım, validation setinde de kesin alman lazım. Eğer boş yerler varsa da bunları 50 ile doldur. Neden train seti 
# medianı kullanıyoruz? Zaten bu veriyi sentetik olarak ekliyoruz. Olabildiğince mantıklı ve iyi olması için uğraşacağız.
# bu yüzden train setinde median'a göre bir mantık kurduk. eğer validation setinde başka değer verirsem, bunu farklı bir mantık olarak
# algılayabilir.

In [26]:
def fix_missing(df, col, name, nan_dict, is_train):
    
    if is_train:
        if is_numeric_dtype(col):
            if pd.isnull(col).sum():
                df[name+"_null"] = pd.isnull(col)
                nan_dict[name] = col.median()
                df[name] = col.fillna(col.median())
                
    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+"_null"] = pd.isnull(col)
                df[name] = col.fillna(nan_dict[name])
            else:
                df[name] = col.fillna(df[name].median())

In [27]:
def proc_df(df, y_fld, nan_dict = None, is_train = True):
    
    df = df.copy()
    y= df[y_fld].values
    
    df.drop([y_fld], axis = 1, inplace = True)
    
    if nan_dict is None:
        nan_dict = {}
    
    
    for n, c in df.items():
        fix_missing(df, c, n, nan_dict, is_train)
        numericalize(df, c, n)
        
    if is_train:
        return df, y, nan_dict
        
    return df, y

In [28]:
#we will have codes starting from 0 (for missing)
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1

In [29]:
def split_train_val(df, n):
    #train i train ve validation olarak bölüyoruz
    return df[:n].copy(), df[n:].copy()

In [30]:
n_valid = 12000 #same as Kaggle's test set size
n_train = len(df)-n_valid #en güncelleri validasyon alıyoruz. gerçek hayata en yakınlar validasyon.
raw_train, raw_valid = split_train_val(df, n_train)

In [31]:
x_train, y_train, nas = proc_df(raw_train, 'SalePrice')

In [32]:
x_valid, y_valid = proc_df(raw_valid, 'SalePrice', nan_dict = nas, is_train = False)

### Model

In [33]:
m = RandomForestRegressor(n_estimators = 1, bootstrap = False, n_jobs = -1) # model iskeletini burada söylüyorum.
#ben senden böyle birşey istiyorum diyorum. ondan sonra verimi veriyorum. Yani sen bu iskeletle benim verimi öğren.
m.fit(x_train, y_train) #burada eğit diyoruz. ve artık veri eğitilmiş oluyor.
m.score(x_train, y_train) #fit olduktan sonra default olarak r^2'i hesaplayacak. bakalım ne kadar başarılı?
#yani bir üstte ne diyor? x_trainden bir y tahmin ettim, ve gerçek y ler var bunları karşılaştırıp başarı oranına bakar

# n_estimators, kaç ağaç kullanacağız onu söylüyor, bootstrap sampling default olarak true. 
# bootstrap kullanak mıyız onu belirtiyor. n_jobs kaç çekirdek kullanıcaz bu işlem için onu 
# belirtiyor. -1 ile hepsini kullan diyoruz. model hızı ile alakalı.

1.0

* burada 1 çıkması belirleyici bir parametre değil. Zira bu sadece train setinde olan. Validasyonda test etmem lazım. zira traindeki overfit oluyo olabilir. arka planda veriyi ezberliyo olabilir. bunu test etmek için validasyonda bakmmam lazım.

In [41]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

In [42]:
def print_score(m):
    print(f"RMSE of train set: {rmse(m.predict(x_train),y_train)}")
    print(f"RMSE of validation set: {rmse(m.predict(x_valid),y_valid)}")
    print(f"R^2 of train set: {m.score(x_train, y_train)}")
    print(f"R^2 of validation set: {m.score(x_valid, y_valid)}")

In [43]:
print_score(m)

RMSE of train set: 0.0
RMSE of validation set: 13189.668767242918
R^2 of train set: 1.0
R^2 of validation set: 0.7289025484253335


* neden değişik çıktı r^2? validation'da biz sorguyu durdurmuyoruz. mesela 1000 tane veri var, en sonda 1000 tane farklı veriseti oluştu sora sora, dallara ayrıla ayrıla. En sonunda her birinde 1 tane veri var. 1000 veri setinin her birinde 1 tane veri var. Çok spesifiğe inebildi. Artık sorular tek bir veriyi ifade edbilecek şekilde sorular en aşağıya kadar indi. böyle olduğu için r^2=1 oldu trainde. validasyonda ise, trainde ezberlediği için sorular spesifik olduğu için hatalar oldu. 
* overfitting'i şöyle engelleyebiliriz: 1- soru sayısını azaltabiliriz. yani bir yerde kesilir random forest. bu yüzden her veri için spesifik sorular sorulmaz. 2- birden fazla ağaç eğitiriz. bu ağaçlar verinin farklı yerlerini görse ve bunların öğrendiklerinin ortalamasını alalım. öğrendikleriini birleştirsek ve yani bir mantığa gidelim. böylece hiç ağaç veriyi ezberlemez.

In [44]:
def get_sample(df, n):
    idxs = np.random.permutation(len(df))[:n]
    return idxs, df.iloc[idxs].copy()

* Validation seti değiştirmek istemiyorum, sadece train içinde subset alacağım.

* 1 milyon verim var diyelim. hepsini kullanıp, birçok model denesem, saatler sürer. bu yüzden subsample alacağım. Model seçerken subsample alacağım ama model kurarken tüm verimi kullanacağım. Ama subsample'ı neye göre seçeceğim?

* 1 milyon içinden 100k subsample alacağım mesela. model-1'de R^2=0.62, model-2'de R^2=0.67, model-3'de R^2=0.63 çıktı. Az çıkması burada önemli değil. Önemli olan, diğerlerine göre model-2 daha iyi sonuç verdi. Ben bu modeli tüm veride kullandığımda da yine diğerlerine göre yüksek R^2 verecek demek.

* Önemli olan subsample'ın, tüm veri karakteristiğini iyi temsil etmesi. bu ne demek? mesela 1m içinden 100k seçtik, R^2 modelde 0.62 çıktı. Başka 100k seçtik, burada da 0.45 çıktı. Başka 100k'da 0.32 çıktı. bir tutarsızlık var. Yani subsample'lar verinin farklı farklı yerlerini görüyor ve veriseti düzgün temsil edemiyor.

* 200k olarak alınsa R^2: 0.62, 0.65 vs. çıkmaya başladı. Daha tutarlı sayılar almaya başladık. Yani bu sample sayısı veriyi iyi ifade edebiliyor. Zira tutarlı R^2'ler alıyoruz.

* Burada subsample seçip (3000 seçmiştik. Bakıyoruz ki bu sayı yeterliymiş), hangi model iyi ona bakıyoruz. Sonra tüm veride bu modeli kullanıyoruz. Aşağıda iki model denendi. biri 10 ağaçlı, diğeri 30 ağaçlı. 30 ağaçlı olan daha iyi sonuç veriyor. Yani tüm veride de 30 ağaçlı iyi sonuç verecek demek.

In [45]:
idxs, x_train = get_sample(x_train, 3000)
y_train = y_train[idxs]

In [46]:
m = RandomForestRegressor(n_estimators = 10, n_jobs = -1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: user 535 ms, sys: 11.4 ms, total: 546 ms
Wall time: 337 ms
RMSE of train set: 5173.714106769268
RMSE of validation set: 14004.459877403788
R^2 of train set: 0.9522062191614833
R^2 of validation set: 0.694373939163849


In [47]:
m = RandomForestRegressor(n_estimators = 30, n_jobs = -1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: user 1.62 s, sys: 19.9 ms, total: 1.64 s
Wall time: 671 ms
RMSE of train set: 4355.1620226356
RMSE of validation set: 12882.54537917479
R^2 of train set: 0.9661331216504893
R^2 of validation set: 0.7413806484120291


### Tüm Veri İle

In [49]:
x_train, y_train, nas = proc_df(raw_train, 'SalePrice')

In [50]:
x_valid, y_valid = proc_df(raw_valid, 'SalePrice', nan_dict = nas, is_train = False)

In [51]:
m = RandomForestRegressor(n_estimators = 10, n_jobs = -1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: user 2min 8s, sys: 1.73 s, total: 2min 10s
Wall time: 59.2 s
RMSE of train set: 3025.1676376188007
RMSE of validation set: 9632.14544068982
R^2 of train set: 0.9826335942383905
R^2 of validation set: 0.8554214982805716


In [52]:
m = RandomForestRegressor(n_estimators = 30, n_jobs = -1)
%time m.fit(x_train, y_train)
print_score(m)

CPU times: user 6min 32s, sys: 3.25 s, total: 6min 35s
Wall time: 2min 17s
RMSE of train set: 2679.282389992527
RMSE of validation set: 9036.941690656698
R^2 of train set: 0.9863777755534994
R^2 of validation set: 0.8727374512669335
